In [ ]:
# | default_exp testing

In [ ]:
# | export


def in_notebook():
    try:
        from IPython import get_ipython

        if "IPKernelApp" not in get_ipython().config:
            return False
    except ImportError:
        return False
    except AttributeError:
        return False
    return True

In [ ]:
in_notebook()

True

In [ ]:
# | export

import asyncio
import contextlib
import hashlib
import os
import random
import shlex
import textwrap

# [B404:blacklist] Consider possible security implications associated with the subprocess module.
import subprocess  # nosec
import time
import unittest
import unittest.mock
from contextlib import asynccontextmanager, contextmanager
from fastcore.meta import delegates
from datetime import datetime, timedelta
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Any, Callable, Dict, Generator, List, Optional, Tuple, AsyncIterator, Union
from pydantic import BaseModel

from aiokafka import AIOKafkaConsumer, AIOKafkaProducer
from confluent_kafka.admin import AdminClient, NewTopic
if in_notebook():
    from tqdm.notebook import tqdm, trange
else:
    from tqdm import tqdm, trange
    
from fast_kafka_api._components.logger import get_logger, supress_timestamps
from fast_kafka_api._components.helpers import delegates_using_docstring
from fast_kafka_api.helpers import create_admin_client, create_missing_topics

In [ ]:
# | notest

from inspect import signature

import nest_asyncio
from nbdev_mkdocs.docstring import run_examples_from_docstring

In [ ]:
# | notest

# allows async calls in notebooks

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export

kafka_server_url = (
    os.environ["KAFKA_HOSTNAME"] if "KAFKA_HOSTNAME" in os.environ else "localhost"
)
kafka_server_port = os.environ["KAFKA_PORT"] if "KAFKA_PORT" in os.environ else "9092"

kafka_config = {
    "bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}",
    # "group.id": f"{kafka_server_url}:{kafka_server_port}_group"
}

In [ ]:
# | export


def true_after(seconds: float) -> Callable[[], bool]:
    """Function returning True after a given number of seconds"""
    t = datetime.now()

    def _true_after(seconds: float = seconds, t: datetime = t) -> bool:
        return (datetime.now() - t) > timedelta(seconds=seconds)

    return _true_after

In [ ]:
f = true_after(1.1)
assert not f()
time.sleep(1)
assert not f()
time.sleep(0.1)
assert f()

In [ ]:
# | export


@contextmanager
@delegates_using_docstring(create_missing_topics)
def create_testing_topic(
    topic_prefix: str, *, seed: Optional[int] = None, **kwargs: Dict[str, Any]
) -> Generator[str, None, None]:
    """Create testing topic
    
    Args:
        topic_prefix: topic name prefix which will be augumented with a randomly generated sufix
        seed: seed used to generate radnom sufix
        
    Returns:
        Generator returning the generated name of the created topic
        
    Example:
        ```python
        from os import environ
        from fast_kafka_api.testing import create_testing_topic, create_admin_client
        
        kafka_server_url = environ["KAFKA_HOSTNAME"]
        kafka_config = {"bootstrap.servers": f"{kafka_server_url}:9092"}
        
        with create_testing_topic("my_topic_", num_partitions=1, **aiokafka_config) as topic:
            # Check if topic is created and exists in topic list
            kafka_admin = create_admin_client(**aiokafka_config)
            existing_topics = kafka_admin.list_topics().topics.keys()
            assert topic in existing_topics

        # Check if topic is deleted after exiting context
        existing_topics = kafka_admin.list_topics().topics.keys()
        assert topic not in existing_topics
        ```
    """
    # create random topic name
    random.seed(seed)
    # [B311:blacklist] Standard pseudo-random generators are not suitable for security/cryptographic purposes.
    suffix = str(random.randint(0, 10**10))  # nosec

    topic = topic_prefix + suffix.zfill(3)

    # delete topic if it already exists
    admin = create_admin_client(**kwargs)
    existing_topics = admin.list_topics().topics.keys()
    if topic in existing_topics:
        logger.warning(f"topic {topic} exists, deleting it...")
        fs = admin.delete_topics(topics=[topic])
        results = {k: f.result() for k, f in fs.items()}
        while topic in admin.list_topics().topics.keys():
            time.sleep(1)
    try:
        # create topic if needed
        create_missing_topics([topic], **kwargs)
        while topic not in admin.list_topics().topics.keys():
            time.sleep(1)
        yield topic

    finally:
        pass
        # cleanup if needed again
        fs = admin.delete_topics(topics=[topic])
        while topic in admin.list_topics().topics.keys():
            time.sleep(1)

In [ ]:
# run_examples_from_docstring(create_testing_topic, width=120, supress_stdout=True)

In [ ]:
kafka_server_url = os.environ["KAFKA_HOSTNAME"]
aiokafka_config = {"bootstrap_servers": f"{kafka_server_url}:9092"}

with create_testing_topic("my_topic_", num_partitions=1, **aiokafka_config) as topic:
    # Check if topic is created and exists in topic list
    kafka_admin = create_admin_client(**aiokafka_config)
    existing_topics = kafka_admin.list_topics().topics.keys()
    assert topic in existing_topics

# Check if topic is deleted after exiting context
existing_topics = kafka_admin.list_topics().topics.keys()
assert topic not in existing_topics


[INFO] fast_kafka_api.helpers: create_missing_topics(['my_topic_6334300722']): new_topics = [NewTopic(topic=my_topic_6334300722,num_partitions=1)]


In [ ]:
# | export


@asynccontextmanager
@delegates(create_testing_topic)
@delegates_using_docstring(AIOKafkaProducer)
async def create_and_fill_testing_topic(
    msgs: List[bytes], *, seed: int, **kwargs: Dict[str, str]
) -> AsyncIterator[str]:
    """
    Args:
        msgs:
        seed:
    """

    with create_testing_topic("my_topic_", seed=seed, **kwargs) as topic:

        producer = AIOKafkaProducer(**kwargs)
        logger.info(f"Producer {producer} created.")

        await producer.start()
        logger.info(f"Producer {producer} started.")
        try:
            fx = [
                producer.send(
                    topic,
                    msg,
                    key=f"{i % 17}".encode("utf-8"),
                )
                for i, msg in enumerate(msgs)
            ]
            await producer.flush()
            sent_msgs = [await f for f in fx]
            msg_statuses = [await s for s in sent_msgs]
            logger.info(f"Sent messages: len(sent_msgs)={len(sent_msgs)}")

            yield topic
        finally:
            await producer.stop()
            logger.info(f"Producer {producer} stoped.")

In [ ]:
msgs_sent = 317
msgs = [f"Hello world {i:05d}".encode("utf-8") for i in range(msgs_sent)]

async with create_and_fill_testing_topic(msgs, seed=1, **aiokafka_config) as topic:
    print("started")

print("ok")

[INFO] fast_kafka_api.helpers: create_missing_topics(['my_topic_9167024629']): new_topics = [NewTopic(topic=my_topic_9167024629,num_partitions=3)]
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> created.
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> started.
[INFO] __main__: Sent messages: len(sent_msgs)=317
started
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> stoped.
ok


In [ ]:
signature(create_and_fill_testing_topic).parameters

In [ ]:
print(create_and_fill_testing_topic.__doc__)

In [ ]:
msgs_sent = 317
msgs = [f"Hello world {i:05d}".encode("utf-8") for i in range(msgs_sent)]

async with create_and_fill_testing_topic(msgs, seed=1, **aiokafka_config) as topic:
    consumer = AIOKafkaConsumer(
        topic,
        bootstrap_servers=kafka_config["bootstrap.servers"],
        auto_offset_reset="earliest",
        max_poll_records=100,
    )
    logger.info(f"Consumer {consumer} created.")
    await consumer.start()
    logger.info(f"Consumer {consumer} started.")
    is_shutting_down_f = true_after(5)
    msgs_received = 0
    try:
        while True:
            msgs = await consumer.getmany(timeout_ms=100)
            for k, v in msgs.items():
                msgs_received = msgs_received + len(v)
            if is_shutting_down_f():
                break

    finally:
        assert msgs_received == msgs_sent
        print(f"Total messages received: {msgs_received}")
        await consumer.stop()
        logger.info(f"Consumer {consumer} stopped.")

[INFO] fast_kafka_api.helpers: create_missing_topics(['my_topic_9167024629']): new_topics = [NewTopic(topic=my_topic_9167024629,num_partitions=3)]
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> created.
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> started.
[INFO] __main__: Sent messages: len(sent_msgs)=317
[INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'my_topic_9167024629'})
[INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> created.
[INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'my_topic_9167024629': 3}. 
[INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> started.
Total messages received: 317
[INFO] __main__: Consumer <aiokafka.consumer.consumer.AIOKafkaConsumer object> stopped.
[INFO] __main__: Producer <aiokafka.producer.producer.AIOKafkaProducer object> stoped.


In [ ]:
assert False

In [ ]:
# TODO: Send repeatedly?

In [ ]:
# | export


def nb_safe_seed(s: str) -> Callable[[int], int]:
    """Gets a unique seed function for a notebook

    Params:
        s: name of the notebook used to initialize the seed function

    Returns:
        A unique seed function
    """
    init_seed = int(hashlib.sha256(s.encode("utf-8")).hexdigest(), 16) % (10**8)

    def _get_seed(x: int = 0, *, init_seed: int = init_seed) -> int:
        return init_seed + x

    return _get_seed

In [ ]:
seed = nb_safe_seed("999_test_utils")

assert seed() == seed(0)
assert seed() + 1 == seed(1)

In [ ]:
# | export


@contextmanager
def mock_AIOKafkaProducer_send() -> Generator[unittest.mock.Mock, None, None]:
    """Mocks **send** method of **AIOKafkaProducer**"""
    with unittest.mock.patch("__main__.AIOKafkaProducer.send") as mock:

        async def _f():
            pass

        mock.return_value = asyncio.create_task(_f())

        yield mock

In [ ]:
# | export


@contextlib.contextmanager
def change_dir(d: str) -> Generator[None, None, None]:
    curdir = os.getcwd()
    os.chdir(d)
    try:
        yield
    finally:
        os.chdir(curdir)

In [ ]:
# TODO: tests
with TemporaryDirectory() as d:
    original_wd = os.getcwd()
    assert original_wd != d
    with change_dir(d):
        assert os.getcwd() == d
    assert os.getcwd() == original_wd

In [ ]:
# | export


def run_script_and_cancel(
    *, script: str, script_file: str, cmd: str, cancel_after: int
) -> Tuple[int, bytes]:
    with TemporaryDirectory() as d:
        consumer_script = Path(d) / script_file

        with open(consumer_script, "a+") as file:
            file.write(script)

        # os.chdir(d)
        with change_dir(d):
            proc = subprocess.Popen(  # nosec: [B603:subprocess_without_shell_equals_true] subprocess call - check for execution of untrusted input.
                shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT
            )
            time.sleep(cancel_after)
            proc.terminate()
            output, _ = proc.communicate()

        return (proc.returncode, output)

In [ ]:
cmd = "python3 -m test.py"

# Check exit code 0
script = "exit(0)"

exit_code, output = run_script_and_cancel(
    script=script, script_file="test.py", cmd=cmd, cancel_after=1
)

assert exit_code == 0
assert output.decode("utf-8") == ""


# Check exit code 1
script = "exit(1)"

exit_code, output = run_script_and_cancel(
    script=script, script_file="test.py", cmd=cmd, cancel_after=1
)

assert exit_code == 1
assert output.decode("utf-8") == ""


# Check exit code 0 and output to stdout and stderr
script = """
import sys
sys.stderr.write("hello from stderr\\n")
sys.stderr.flush()
print("hello, exiting with exit code 0")
exit(0)
"""

exit_code, output = run_script_and_cancel(
    script=script, script_file="test.py", cmd=cmd, cancel_after=1
)

assert exit_code == 0, exit_code
assert output.decode("utf-8") == "hello from stderr\nhello, exiting with exit code 0\n", output.decode("utf-8")


# Check random exit code and output
script = """
print("hello\\nexiting with exit code 143")
exit(143)
"""

exit_code, output = run_script_and_cancel(
    script=script, script_file="test.py", cmd=cmd, cancel_after=1
)

assert exit_code == 143
assert output.decode("utf-8") == "hello\nexiting with exit code 143\n"

print("ok")